In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19

In [2]:
vgg_model = VGG19(weights='imagenet' , include_top = False , input_shape=(224,224,3))

2024-05-25 19:26:18.854779: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-05-25 19:26:18.854804: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-05-25 19:26:18.854810: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-05-25 19:26:18.855351: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-25 19:26:18.855796: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
dataset_url = r'/Users/aditya/Desktop/General/Projects/2750'
batch_size = 32
img_height = 244
img_width = 244
validation_split=0.2
rescale=1.0/255

In [4]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=validation_split, rescale=rescale)
dataset = tf.keras.preprocessing.image_dataset_from_directory(dataset_url, image_size=(img_height, img_width), batch_size=batch_size)

Found 27000 files belonging to 10 classes.


In [5]:
train_dataset = datagen.flow_from_directory(batch_size=batch_size,
                                           directory=dataset_url,
                                           shuffle=True,
                                           target_size=(img_height, img_width),
                                           subset="training",
                                           class_mode='categorical')

Found 21600 images belonging to 10 classes.


In [6]:
test_dataset = datagen.flow_from_directory(batch_size=batch_size,
                                           directory=dataset_url,
                                           shuffle=True,
                                           target_size=(img_height, img_width),
                                           subset="validation",
                                           class_mode='categorical')

Found 5400 images belonging to 10 classes.


In [7]:
model = tf.keras.Sequential()
model.add(vgg_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(96, activation='relu'))
model.add(tf.keras.layers.Dropout(.25))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [8]:
for layer in vgg_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=2)
with tf.device("/gpu:0"):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']  )
    history = model.fit(train_dataset, validation_data=test_dataset, epochs=10, batch_size=32 , callbacks = [callback])

Epoch 1/10


2024-05-25 19:26:20.598880: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


675/675 [==============================] - 997s 1s/step - loss: 1.2238 - accuracy: 0.5824 - val_loss: 0.8467 - val_accuracy: 0.7117
Epoch 2/10
675/675 [==============================] - 998s 1s/step - loss: 0.8486 - accuracy: 0.7105 - val_loss: 0.7343 - val_accuracy: 0.7583
Epoch 3/10
675/675 [==============================] - 1063s 2s/step - loss: 0.7518 - accuracy: 0.7428 - val_loss: 0.6926 - val_accuracy: 0.7641
Epoch 4/10
675/675 [==============================] - 1289s 2s/step - loss: 0.7213 - accuracy: 0.7479 - val_loss: 0.6667 - val_accuracy: 0.7650
Epoch 5/10
675/675 [==============================] - 1284s 2s/step - loss: 0.6818 - accuracy: 0.7613 - val_loss: 0.5932 - val_accuracy: 0.7926
Epoch 6/10
675/675 [==============================] - 1209s 2s/step - loss: 0.6463 - accuracy: 0.7746 - val_loss: 0.6427 - val_accuracy: 0.7776
Epoch 7/10
675/675 [==============================] - 1080s 2s/step - loss: 0.6270 - accuracy: 0.7793 - val_loss: 0.5766 - val_accuracy: 0.7939
Epoch

In [ ]:
model.save('lulc_10_epoch_vgg19.h5')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))
ax1.plot(history.history['accuracy'])
ax1.plot(history.history['val_accuracy'])
ax1.set_xlabel('epoch')
ax1.set_ylabel('accuracy')
ax1.set_title('Accuracy over epoch')
ax1.legend(['Train', 'Test'], loc='upper left')
ax1.grid(True)
ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_xlabel('epoch')
ax2.set_ylabel('loss')
ax2.set_title('Loss over epoch')
ax2.legend(['Train', 'Test'], loc="upper right")
ax2.grid(True)

In [ ]:
preds = model.predict(test_dataset)